In [3]:
import numpy as np

data_careem = np.load("D:\omer\Spatial_Master_Grid_15x19(60_min_resolution).npy", allow_pickle=True)

print("data_careem.shape: ", data_careem.shape)
data_NYC = np.load('D:\\omer\\ST-SSL\\data\\NYCBike1\\train.npz')
for file in data_NYC.files:
    print(file, data_NYC[file].shape)

data_careem.shape:  (2160, 15, 19, 1)
x (3023, 19, 128, 2)
y (3023, 1, 128, 2)
x_offsets (19, 1)
y_offsets (1, 1)


In [85]:
indices = data_NYC["x_offsets"]
print(indices)

[[-73]
 [-72]
 [-71]
 [-70]
 [-69]
 [-49]
 [-48]
 [-47]
 [-46]
 [-45]
 [-25]
 [-24]
 [-23]
 [-22]
 [-21]
 [ -3]
 [ -2]
 [ -1]
 [  0]]


In [4]:
reshaped_data_careem.shape

NameError: name 'reshaped_data_careem' is not defined

The train/val/test data is composed of 4 numpy.ndarray objects:

X: input data. It is a 4D tensor of shape (#samples, #lookback_window, #nodes, #flow_types), where # denotes the number sign.
<br>Y: data to be predicted. It is a 4D tensor of shape (#samples, #predict_horizon, #nodes, #flow_types). Note that X and Y are paired in the sample dimension. For instance, (X_i, Y_i) is the i-the data sample with i indexing the sample dimension.
<br>X_offset: a list indicating offsets of X's lookback window relative to the current time with offset 0.
<br>Y_offset: a list indicating offsets of Y's prediction horizon relative to the current time with offset 0.

In [5]:
import networkx as nx
import numpy as np

# Create a 15x19 grid graph
G = nx.grid_2d_graph(15, 19)

# Create an empty adjacency matrix
adj_matrix = np.zeros((15*19, 15*19))

# Iterate over all nodes in the grid
for node in G.nodes:
    # Get the row and column indices of the current node
    row, col = node
    
    # Iterate over the 8 neighbors of the current node
    for i in range(-1, 2):
        for j in range(-1, 2):
            # Skip the current node itself
            if i == 0 and j == 0:
                continue
            
            # Calculate the row and column indices of the neighbor
            neighbor_row = row + i
            neighbor_col = col + j
            
            # Check if the neighbor is within the grid boundaries
            if 0 <= neighbor_row < 15 and 0 <= neighbor_col < 19:
                # Calculate the indices of the current node and the neighbor in the adjacency matrix
                node_index = row * 19 + col
                neighbor_index = neighbor_row * 19 + neighbor_col
                
                # Set the corresponding entry in the adjacency matrix to 1
                adj_matrix[node_index, neighbor_index] = 1

# Print the adjacency matrix
print(adj_matrix)


[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]


In [50]:
np.savez('D:\\omer\\ST-SSL\\data\\Careem\\adj_matrix.npz', adj_mx=adj_matrix)


In [6]:
reshaped_data_careem = data_careem.reshape(data_careem.shape[0], -1, 1)
print("reshaped_data_careem.shape: ", reshaped_data_careem.shape)  ## #samples, #nodes, demand


reshaped_data_careem.shape:  (2160, 285, 1)


In [7]:
reshaped_data_careem_2 = np.expand_dims(reshaped_data_careem, axis=1)
print("reshaped_data_careem_2.shape: ", reshaped_data_careem_2.shape)  ## #samples, #lookback_window #nodes, demand

reshaped_data_careem_2.shape:  (2160, 1, 285, 1)


In [34]:
print(reshaped_data_careem_2[-5, 0, :, :].shape)
reshaped_data_careem_2[-2, 0, :, :]

(285, 1)


array([[  0.],
       [  0.],
       [  4.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  1.],
       [  0.],
       [  1.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [ 60.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  1.],
       [  2.],
       [  1.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  1.],
       [  0.],
       [  0.],
       [ 12.],
       [  0.],
       [  2.],
       [  0.],
       [  2.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  2.],
       [  1.],
       [  0.],
       [  0.],
       [  0.],
       [105.],
       [  2.],
       [  2.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  4.],
       [  1.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  

In [68]:
import torch
tensor1 = torch.tensor([[[[1, 1], [2, 2], [3, 3], [4, 4]]]], dtype=torch.float32) # Shape: [1, 1, 4, 1]
tensor2 = torch.tensor([[[[5, 3]]]], dtype=torch.float32)                # Shape: [1, 1, 1, 1]

print(tensor1.shape, tensor1[:, :, 2, :].shape, tensor2.shape)
# Replacing the 3rd element (index 2) of the 3rd dimension of tensor1 with tensor2
tensor1[:, :, 2, :] = tensor2

tensor1

torch.Size([1, 1, 4, 2]) torch.Size([1, 1, 2]) torch.Size([1, 1, 1, 2])


tensor([[[[1., 1.],
          [2., 2.],
          [5., 3.],
          [4., 4.]]]])

In [10]:
indices = [[i] for i in range(-3, 1)]


In [13]:
## making the lookback window

import numpy as np

# Assuming original_data is your original tensor with shape (2160, 1, 285, 1)
print(indices)
# Initialize the target tensor with zeros
careem_X = np.zeros((2160-4, 4, 285, 1))
careem_Y = np.zeros((2160-4, 1, 285, 1))
# Iterate over each sample
for i in range(3, 2160-1):
    # For each sample, fetch the indices from index_list
    
    
    # Iterate over each index to fill the target tensor
    for j, index in enumerate(indices):
        index=index[0]
        # print(index)
        # Copy the data from the original tensor to the target tensor
        careem_X[i-3, j, :, :] = reshaped_data_careem_2[i+index, 0, :, :]
    careem_Y[i-3, 0, :, :] = reshaped_data_careem_2[i+1, 0, :, :]


[[-3], [-2], [-1], [0]]


In [81]:
# arr1 = np.array([[[1, 2], [3, 4]]])
# arr2 = np.array([[5, 6]])
# print(arr1)
# print(arr1.shape)
# arr1[:, :, -1] = arr2
# print(arr1)
# print(arr1.shape, arr2.shape)

In [14]:
print("careem_X.shape: ", careem_X.shape)
print("careem_Y.shape: ", careem_Y.shape)

careem_X.shape:  (2156, 4, 285, 1)
careem_Y.shape:  (2156, 1, 285, 1)


In [15]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets (70:30 split)
X_train, X_test, Y_train, Y_test = train_test_split(careem_X, careem_Y, test_size=0.3, random_state=42)

# Split the test set into test and validation sets (50:50 split)
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.33, random_state=42)

# Print the shapes of the train, test, and validation sets
print("X_train.shape:", X_train.shape)
print("Y_train.shape:", Y_train.shape)
print("X_test.shape:", X_test.shape)
print("Y_test.shape:", Y_test.shape)
print("X_val.shape:", X_val.shape)
print("Y_val.shape:", Y_val.shape)


X_train.shape: (1509, 4, 285, 1)
Y_train.shape: (1509, 1, 285, 1)
X_test.shape: (433, 4, 285, 1)
Y_test.shape: (433, 1, 285, 1)
X_val.shape: (214, 4, 285, 1)
Y_val.shape: (214, 1, 285, 1)


In [16]:
np.savez('D:\\omer\\ST-SSL\\data\\Careem\\train.npz', x=X_train, y=Y_train)
np.savez('D:\\omer\\ST-SSL\\data\\Careem\\test.npz', x=X_test, y=Y_test)
np.savez('D:\\omer\\ST-SSL\\data\\Careem\\val.npz', x=X_val, y=Y_val)


In [17]:
prediction_buffer = None

In [26]:

pred_output = 4
# Update the prediction buffer
if prediction_buffer is None:
    prediction_buffer = [pred_output]
else:
    prediction_buffer.insert(0, pred_output)
    prediction_buffer = prediction_buffer[:4]  # Keep buffer size same as the number of time steps



In [27]:
prediction_buffer

[4, 3, 2, 1]